In [1]:
import requests
import pickle
import time
import json

Setting the proper parameters to get the best-fitting results from google maps

In [2]:
request_count = 1  # max amount of requests sent to google maps
data = {}  # data consists of the following tupples: placed_id -> (name, types, lat, lng)
parameters = {
    'location': str(50.0615868) + ',' + str(19.9373393),  # krakow old town 
    'radius': 10000,  # 10 km
    'language': 'en',  # the result language
    'key': ''  # the google maps api key of the app
}

The method executes the request to google maps

In [3]:
def execute_request(parameters):
    return requests.get('https://maps.googleapis.com/maps/api/place/nearbysearch/json', params=parameters)

The method processes the result by extracting needed data

In [4]:
def extract_data(response):
    data = {}
    places = response.json()['results']
    for place in places:
        place_id = place["place_id"]
        name = place["name"]
        types = place["types"]
        lat = place['geometry']['location']["lat"]
        lng = place['geometry']['location']["lng"]
        data[place_id] = (name, types, lat, lng)
    return data

The loop that gathers the data

In [5]:
for _ in range(request_count):
    response = execute_request(parameters)
    response_data = extract_data(response)
    for key, value in response_data.items():
        if key not in data:
            data[key] = value
    print("Response data len:", len(response_data))

print("Total data:", len(data)) 

Response data len: 20
Total data: 20


Convert dict to list

In [6]:
result = [(key, value[0], value[1], value[2], value[3]) for key, value in data.items()]

Save the result to the pickle file

In [7]:
with open(f'../data/data_{int(time.time())}.pickle', 'wb') as handle:
    pickle.dump(result, handle)